In [1]:
from pathlib import Path
import sys
import os

import gtfs_kit as gk
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium as fl
import dotenv as de

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

de.load_dotenv(de.find_dotenv())


True

# Load API keys from environment variables

In [2]:
mapbox_key = os.getenv('MAPBOX_KEY')
google_key = os.getenv('GOOGLE_KEY')

# Load sample GTFS feed

In [3]:
path = mm.DATA_DIR/'auckland_gtfs_sample.zip'
feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Go West, AT Metro, Fullers Ferries, Pavlovich..."
1,timezone,Pacific/Auckland
2,start_date,20171012
3,end_date,20171220
4,num_routes,15
5,num_trips,4675
6,num_stops,1022
7,num_shapes,195
8,sample_date,20171019
9,num_routes_active_on_sample_date,15


# Create sample points

In [4]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[-1].values

points_and_keys_by_method = {
    "stop": mm.sample_trip_points(feed, [trip_id])[0],
    "distance":  mm.sample_trip_points(feed, [trip_id], method='distance', value=0.1)[0],
    "num_points": mm.sample_trip_points(feed, [trip_id], method='num_points', value=100)[0],
}



# Compare sample points with one actual feed shape

In [5]:
line_by_method = {
    method: sg.LineString(points)
    for method, (points, key) in points_and_keys_by_method.items()
}

# Get actual trip shape
p = mm.get_stop_patterns(feed)
line_by_method["actual"] = feed.build_geometry_by_shape(shape_ids=[shape_id])[shape_id]

# Plot
center = points_and_keys_by_method["stop"][0][0][::-1]
fl_map = fl.Map(location=center, zoom_start=14, tiles="cartodbpositron")
colors = ["red", "blue", "yellow", "green"]
for i, (method, line) in enumerate(line_by_method.items()):
    fg = fl.FeatureGroup(name=method)
    fl.PolyLine([(p[1], p[0]) for p in line.coords], color=colors[i]).add_to(fg)
    fg.add_to(fl_map)
    
fl.LayerControl().add_to(fl_map)
fl_map

# Match stops of one stop pattern using the various matching services and compare to GTFS shape

In [6]:
spoints, id_ = points_and_keys_by_method["distance"]

In [14]:
line_by_service = {"sample points": sg.LineString(spoints)}
for service, fn, key in [
    ("osrm", mm.match_with_osrm, None),
    ("mapbox", mm.match_with_mapbox, mapbox_key),
    ("google", mm.match_with_google, google_key),
]:
    print(service)
    %time mpoints, __ = fn([(spoints, id_)], api_key=key)[0]
    line_by_service[service] = sg.LineString(mpoints)

fl_map = fl.Map(location=center, zoom_start=14, tiles="cartodbpositron")
colors = ["black", "orange", "blue", "lightgreen"]
for i, (service, line) in enumerate(line_by_service.items()):
    fg = fl.FeatureGroup(name=service)
    fl.PolyLine([(p[1], p[0]) for p in line.coords], color=colors[i]).add_to(fg)
    fg.add_to(fl_map)
    
fl.LayerControl().add_to(fl_map)
fl_map

osrm
CPU times: user 11.2 ms, sys: 3.98 ms, total: 15.2 ms
Wall time: 1.39 s
mapbox
CPU times: user 29.2 ms, sys: 0 ns, total: 29.2 ms
Wall time: 438 ms
google
CPU times: user 27.6 ms, sys: 4.14 ms, total: 31.7 ms
Wall time: 1.07 s


# Match entire feed

More specifically, match the feed shapes of on-road routes, the default setting

In [8]:
print('Num service calls needed =', mm.get_num_match_calls(feed))

%time feed_2 = mm.match_feed(feed, 'mapbox', mapbox_key)

Num service calls needed = 50
CPU times: user 2.48 s, sys: 63.9 ms, total: 2.55 s
Wall time: 15 s


In [9]:
# Number of shapes should be the same between feeds

feed_2.shapes.shape_id.nunique() == feed.shapes.shape_id.nunique()

True

In [15]:
# Spot check one stop pattern and shape

t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[-1].values


geom_by_shape_1 = feed.build_geometry_by_shape(shape_ids=[shape_id], use_utm=False)
g_1 = geom_by_shape_1[shape_id]
geom_by_shape_2 = feed_2.build_geometry_by_shape(shape_ids=[shape_id], use_utm=False)
g_2 = geom_by_shape_2[shape_id]

fl_map = fl.Map(location=g_1.coords[0][::-1], zoom_start=14, tiles="cartodbpositron")

fg = fl.FeatureGroup(name="original")
fl.PolyLine([(p[1], p[0]) for p in g_1.coords], color="black").add_to(fg)
fg.add_to(fl_map)

fg = fl.FeatureGroup(name="matched")
fl.PolyLine([(p[1], p[0]) for p in g_2.coords], color="orange").add_to(fg)
fg.add_to(fl_map)

fl.LayerControl().add_to(fl_map)

fl_map
